# Displaying image data in TensorBoard

## Overview

使用 TensorFlow Image Summary API，您可以轻松记录张量和任意图像，并在 TensorBoard 中查看它们。 这对于采样、检查您的输入数据，或可视化层权重和生成的张量非常有帮助。您还可以将诊断数据记录为图像，这有助于您的模型开发过程。

在本教程中，您将学习如何使用 Image Summary API 将张量可视化为图像。您还将学习如何拍摄任意图像，将其转换为张量，并在TensorBoard中可视化。您将通过一个简单但真实的示例，该示例使用  Image Summaries 来帮助您了解模型的表现。

## Setup

In [1]:
try:
  # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [3]:
from datetime import datetime
import io
import itertools
from packaging import version

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.6.0


## Download the Fashion-MNIST dataset

您将构建一个简单的神经网络，在 Fashion-MNIST 数据集中对图像进行分类。该数据集包括来自10个类别的70,000张28x28时尚产品灰度图像，每个类别7,000张图像。

首先下载数据：

In [ ]:
# Download the data. The data is already divided into train and test.
# The labels are integers representing classes.
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = \
    fashion_mnist.load_data()

# Names of the integer classes, i.e., 0 -> T-short/top, 1 -> Trouser, etc.
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Visualizing a single image

要了解  Image Summary  API 的工作原理，您现在只需在 TensorBoard 中记录训练集中的第一个训练图像。

在此之前，请检查训练数据的形状：

In [5]:
print("Shape: ", train_images[0].shape)
print("Label: ", train_labels[0], "->", class_names[train_labels[0]])

Shape:  (28, 28)
Label:  9 -> Ankle boot


请注意，数据集中每张图像是形状为(28，28)的 rank-2 张量，代表高度和宽度。

然而，`tf.summary.image()` 期望一个包含 `(batch_size、height、width、channels)` 的 rank-4 张量。因此，张量需要被 reshape。

您只记录了一张图像，因此 `batch_size` 为 1。图像是灰度的，所以将通道设置为1。

In [6]:
# Reshape the image for the Summary API.
img = np.reshape(train_images[0], (-1, 28, 28, 1))

您现在可以记录此图像并在 TensorBoard 中查看。

In [8]:
# Clear out any prior log data.
!rm -rf logs

# Sets up a timestamped log directory.
logdir = "logs/train_data/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the reshaped image.
with file_writer.as_default():
    tf.summary.image("Training data", img, step=0)

Metal device set to: Apple M1


2022-04-04 17:33:18.506556: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-04 17:33:18.507551: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


现在，使用 TensorBoard 检查图像。

In [9]:
%tensorboard --logdir logs/train_data

“Images” 选项卡显示您刚刚记录的图像。这是一只 “ankle boot”。

图像缩放为默认大小，以便于查看。如果您想查看未缩放的原始图像，请检查左上角的 “Show actual image size”。

拖动亮度和对比度滑块，看看它们如何影响图像像素。

## Visualizing multiple images

记录一个张量很棒，但是如果您想记录多个训练样本呢？

只需指定在将数据传递给 `tf.summary.image()`.

In [10]:
with file_writer.as_default():
    # Don't forget to reshape.
    images = np.reshape(train_images[0:25], (-1, 28, 28, 1))
    tf.summary.image("25 training data examples", images, max_outputs=25, step=0)

%tensorboard --logdir logs/train_data

Reusing TensorBoard on port 6006 (pid 14415), started 0:03:20 ago. (Use '!kill 14415' to kill it.)

## Logging arbitrary image data

如果您想可视化非张量的图像，例如由 matplotlib 生成的图像，该怎么办？

你需要一些样板代码来将 plot 转换为张量，但之后，你可以走了。

在下面的代码中，您将使用 matplotlib 的 `subplot()` 函数将前25张图像记录为一个不错的网格。然后，您将在 TensorBoard 中查看网格：

In [11]:
# Clear out prior logging data.
!rm -rf logs/plots

logdir = "logs/plots/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

def image_grid():
    """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
    # Create a figure to contain the plot.
    figure = plt.figure(figsize=(10,10))
    for i in range(25):
        # Start next subplot.
        plt.subplot(5, 5, i + 1, title=class_names[train_labels[i]])
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(train_images[i], cmap=plt.cm.binary)
    return figure

# Prepare the plot
figure = image_grid()
# Convert to image and log
with file_writer.as_default():
    tf.summary.image("Training data", plot_to_image(figure), step=0)

%tensorboard --logdir logs/plots

## Building an image classifier

现在把这一切与一个真实的例子放在一起。毕竟，你来这里是为了做机器学习，而不是绘制漂亮的图片！

您将使用 image summaries 来了解您的模型在为Fashion-MNIST数据集训练一个简单的分类器时的表现。

首先，创建一个非常简单的模型并编译它，设置优化器和损失函数。编译步骤还指定您希望在此过程中记录分类器的准确性。

In [12]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

在训练分类器时，查看混淆矩阵很有用。混淆矩阵让您详细了解分类器在测试数据上的表现。

定义一个计算混淆矩阵的函数。您将使用方便的 Scikit-learn 函数来执行此操作，然后使用 matplotlib 绘制。

In [13]:
def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Compute the labels from the normalized confusion matrix.
    labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

您现在可以训练分类器，并在此过程中定期记录混淆矩阵。

您将完成以下操作：

- 创建 Keras TensorBoard 回调以记录基本指标
- 创建一个Keras LambdaCallback，在每个 epoch 结束时记录混淆矩阵
- 使用 `Model.fit()` 训练模型，确保传入两个回调

随着训练的进行，向下滚动以查看 TensorBoard 启动。

In [15]:
# Clear out prior logging data.
!rm -rf logs/image

logdir = "logs/image/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Define the basic TensorBoard callback.
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

2022-04-04 17:53:16.661763: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-04-04 17:53:16.661787: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-04-04 17:53:16.662977: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


In [16]:
def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

# Define the per-epoch callback.
cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [17]:
# Start TensorBoard.
%tensorboard --logdir logs/image

# Train the classifier.
model.fit(
    train_images,
    train_labels,
    epochs=5,
    verbose=0, # Suppress chatty output
    callbacks=[tensorboard_callback, cm_callback],
    validation_data=(test_images, test_labels),
)

2022-04-04 18:01:22.431442: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-04 18:01:22.434788: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-04 18:01:22.591048: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-04-04 18:01:22.736190: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-04-04 18:01:22.736201: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-04-04 18:01:22.741304: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-04-04 18:01:22.744011: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-04-04 18:01:22.747753: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/im

请注意，训练和验证集的准确性都在攀升。这是个好兆头。但是，该模型在数据的特定子集上的表现如何？

选择 “Image” 选项卡以可视化您记录的混淆矩阵。检查左上角的“Show actual image size”，以查看完整大小的混淆矩阵。

默认情况下，dashboard 显示最后一个记录步骤或 epoch 的 image summary。使用 slider 查看早期的混淆矩阵。注意矩阵如何随着训练的进展而发生重大变化，较暗的正方形沿着对角线合并，而矩阵的其余部分趋向0和白色。这意味着随着训练的进展，您的分类器正在改进！干得好！

混淆矩阵表明，这个简单的模型存在一些问题。尽管取得了巨大进步，但衬衫、T恤和套头衫正在相互混淆。该模型需要更多的工作。

如果您有兴趣，请尝试使用卷积网络（CNN）改进此模型。